In [1]:
import requests

# FIXME: Should implement ourselves
#   https://huggingface.co/spaces/yangheng/PyABSA/blob/main/app.py#L175
def absa(text):

    data = {
        "text": text,
        "dataset": "Laptop14",
        "task": "ACOS",
    }

    try:
        response = requests.post("https://pyabsa.pagekite.me/api/inference", json=data)
        response.raise_for_status()
        
        result = response.json()
        
        return result["Quadruples"]

    except Exception as e:
        print(e)
        return []

In [2]:
from memory.cozo import client

In [3]:
from memory.cozo_relations import create_all
create_all()

In [4]:
from uuid import uuid4
uuid = lambda: str(uuid4())

In [5]:
samantha_character_id = 'fa9dbe95-9a17-49d9-96d3-764f1970b32f' or uuid()
diwank_character_id = 'd5f6a824-efbd-4945-b267-d9cd12cada77' or uuid()

print(dict(
    samantha_character_id=samantha_character_id,
    diwank_character_id=diwank_character_id,
))

{'samantha_character_id': 'fa9dbe95-9a17-49d9-96d3-764f1970b32f', 'diwank_character_id': 'd5f6a824-efbd-4945-b267-d9cd12cada77'}


In [6]:
# False and client.run(f"""
client.run(f"""
?[character_id, name, is_human, about, model] <- [
  ["{samantha_character_id}", "Samantha", false, "Samantha is a personal assistant, she is very diligent, incredible at her job and really really funny.", "julep-ai/samantha-33b"],
  ["{diwank_character_id}", "Diwank", true, "Diwank is an AI researcher, entrepreneur and an amazing person.", null],
]

:put characters {{
    character_id, name, is_human, about, model
}}
""")

,status
0,OK


In [7]:
from memory.embed import embed

In [8]:
import json

with open("./diwank-samantha-principles-all.json", 'r') as f:
    beliefs = json.load(f)
    beliefs = beliefs["beliefs"]

In [9]:
cid_map = dict(
    samantha=samantha_character_id,
    diwank=diwank_character_id,
)

In [77]:
from textwrap import dedent
import re

from tqdm.notebook import tqdm

rm_newline = lambda s: s.replace('\n', '    ').replace('"', "'")

for b in tqdm(beliefs):
    # break

    aspects = absa(b["belief"])
    # aspects = []
    aspects_str = json.dumps(aspects)
    
    query = dedent(f"""
    ?[belief_id, character_id, belief, details, parent_belief_id, valence, aspects, belief_embedding, details_embedding] <- [[
      to_uuid("{b['belief_id']}"),
      to_uuid("{cid_map[b['character_name'].lower()]}"),
      "{rm_newline(b['belief'])}",
      "{rm_newline(b['details'])}",
      {'"'+b['parent_belief_id']+'"' if b['parent_belief_id'] else 'null'},
      {b['valence']},
      {aspects_str},
      vec({b['belief_embedding']}),
      vec({b['details_embedding']}),
    ]]
    
    :put beliefs {{
        belief_id, character_id, belief, details, parent_belief_id, valence, belief_embedding, details_embedding
    }}
    """)

    try:
        client.run(query)
    except Exception as e:
        e = str(e)
        idx = int(re.findall(r"[\d]+", e)[0])
        print(e)
        print()
        print("="*20)
        print()
        print(query[idx-10:idx+20])

  0%|          | 0/768 [00:00<?, ?it/s]

In [14]:
email = "diwank@julep.ai"
character_id = diwank_character_id
assistant_id = samantha_character_id

def add_user(email: str, character_id: str, assistant_id: str):
    return client.run(dedent(f"""
    ?[email, character_id, assistant_id] <- [
      ["{email}", "{character_id}", "{assistant_id}"]
    ]
    
    :put users {{
        email, character_id, assistant_id
    }}
    """))

add_user(email, character_id, assistant_id)

,status
0,OK


In [15]:
def get_character_about(character_id: str):
    
    results = client.run(dedent(f"""
    char[character_id] <- [[to_uuid("{character_id}")]]
    ?[name, about, metadata] := char[character_id], *characters{{name, about, metadata, character_id, updated_at @ floor(now())}}
    """))

    name = results["name"][0]
    about = results["about"][0]
    metadata = results["metadata"][0]

    return dict(
        name=name,
        about=about,
        metadata=metadata,
    )

get_character_about(samantha_character_id)

{'name': 'Samantha',
 'about': 'Samantha is a personal assistant, she is very diligent, incredible at her job and really really funny.',
 'metadata': {}}

In [16]:
def get_user_by_email(email: str):
    results = client.run(dedent(f"""
    input[email] <- [["{email}"]]
    ?[character_id, assistant_id] := input[email], *users{{email, character_id, assistant_id}}
    """))

    character_id = results["character_id"][0]
    assistant_id = results["assistant_id"][0]

    output = dict(
        character_id=character_id,
        assistant_id=assistant_id,
        assistant_about=get_character_about(character_id),
        character_about=get_character_about(assistant_id),
    )

    return output

get_user_by_email("diwank@julep.ai")

{'character_id': 'd5f6a824-efbd-4945-b267-d9cd12cada77',
 'assistant_id': 'fa9dbe95-9a17-49d9-96d3-764f1970b32f',
 'assistant_about': {'name': 'Diwank',
  'about': 'Diwank is an AI researcher, entrepreneur and an amazing person.',
  'metadata': {}},
 'character_about': {'name': 'Samantha',
  'about': 'Samantha is a personal assistant, she is very diligent, incredible at her job and really really funny.',
  'metadata': {}}}

In [17]:
def get_sessions_by_email(email: str, vocode_conversation_id: str, limit: int = 1):
    user_details = get_user_by_email("diwank@julep.ai")
    character_id = user_details["character_id"]
    
    results = client.run(dedent(f"""
    char[character_id, vocode_conversation_id] <- [
      [to_uuid("{character_id}"), "{vocode_conversation_id}"]
    ]
                         
    ?[session_id, situation, summary, metadata] := char[character_id, vocode_conversation_id], *session_characters{{session_id, character_id}}, *sessions{{session_id, situation, summary, metadata, updated_at @ floor(now())}}, vocode_conversation_id == maybe_get(metadata, "vocode_conversation_id")
    :limit {limit}
    """))

    if not len(results):
        return None

    session_id = results["session_id"][0]
    situation = results["situation"][0]
    summary = results["summary"][0]
    metadata = results["metadata"][0]

    return dict(
        session_id=session_id,
        situation=situation,
        summary=summary,
        metadata=metadata,
    )

get_sessions_by_email("diwank@julep.ai", "")

In [18]:
def create_session(email: str, vocode_conversation_id: str, situation: str):
    user_details = get_user_by_email("diwank@julep.ai")
    character_id = user_details["character_id"]
    assistant_id = user_details["assistant_id"]
    
    session_id = uuid()
    metadata = dict(vocode_conversation_id=vocode_conversation_id)
    metadata_str = json.dumps(metadata)

    session_results = client.run(dedent(f"""
    ?[session_id, situation, metadata] <- [[
        to_uuid("{session_id}"),
        "{situation}",
        {metadata_str},
    ]]
    :put sessions {{
        session_id, situation, metadata
    }}
    """))
    
    session_characters_results = client.run(dedent(f"""
    ?[session_id, character_id] <- [
        [to_uuid("{session_id}"), to_uuid("{character_id}")],   
        [to_uuid("{session_id}"), to_uuid("{assistant_id}")],   
    ]
    :put session_characters {{
        session_id, character_id
    }}
    """))

    return session_id

# create_session("diwank@julep.ai", "gagagagaga", "nm")

In [19]:
get_sessions_by_email("diwank@julep.ai", "gagagagaga")

In [20]:
from typing import Literal, Optional, TypedDict

class ChatMLMessage(TypedDict):
    name: Optional[str] = None
    role: Literal["assistant", "system", "user"]
    content: str

ChatML = list[ChatMLMessage]

def message_role_to_prefix(message: ChatMLMessage) -> str:
    match message:
        case {"role": "system", "name": name, **rest}:
            return name
        case {"role": "user", "name": name, **rest}:
            return f"person ({name})" if name else "person"
        case {"role": "assistant", "name": name, **rest}:
            return f"me ({name})" if name else "me"


def to_prompt(
    messages: ChatML,
    suffix: str,
    bos: str = "<|section|>",
    eos: str = "<|endsection|>",
) -> str:
    # Input format:
    # [
    #     {"role": "system", "name": "situation", "content": "I am talking to Diwank"},
    #     {"role": "assistant", "name": "Samantha", "content": "Hey Diwank"},
    #     {"role": "user", "name": "Diwank", "content": "Hey!"},
    # ]
    
    # Output format:
    #
    # <|section|>situation
    # I am talking to Diwank<|endsection|>
    # <|section|>me (Samantha)
    # Hey Diwank<|endsection|>
    # <|section|>person (Diwank)
    # Hey<|endsection|>
    # <|section|>me (Samantha)\n
    

    prompt = "\n".join([
        f"{bos}{message_role_to_prefix(message)}\n{message['content']}{eos}"
        for message in messages
    ])

    return prompt + suffix

In [21]:
from functools import lru_cache
from transformers import AutoTokenizer

tokenizer_id = "timdettmers/guanaco-65b-merged"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, use_fast=False)

@lru_cache
def count_tokens(prompt: str):
    tokens = tokenizer.encode(prompt)
    return len(tokens)

In [22]:
chatml = [{"role": "user", "content": "Hello "*22}]
prompt = to_prompt(chatml, suffix="<|section|>me (Samantha)\n")
count_tokens(prompt)

47

In [23]:
# TODO: IMPLEMENT CACHE USING COZO

In [24]:
def get_beliefs_of_character(character_id: str, limit: int = 100):
    query = dedent(f"""
    char[character_id] <- [[to_uuid("{diwank_character_id}")]]
    ?[belief_id] := *beliefs{{belief_id, character_id}}, char[character_id]

    :limit {limit}
    """)

    return query

In [25]:
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo

def get_aligned_datetime(
    time: datetime,
    align_by: timedelta = timedelta(minutes=30),
    tz: timezone = timezone.utc,
) -> datetime:
    time = time.astimezone(tz).replace(tzinfo=None)
    r = (datetime.min - time) % align_by
    result = (
        time - (time - datetime.min) % align_by
        if r.seconds > align_by.seconds / 2
        else time + r
    )

    return result.replace(tzinfo=tz)

def get_human_data_time(tz: timezone = ZoneInfo('Asia/Kolkata')) -> str:
    """To get the current date and time"""
    dt = get_aligned_datetime(datetime.now(), tz=tz)

    return dt.strftime("%I:%M%p on %A, %b %d %Y")

In [26]:
get_human_data_time()

'09:30AM on Tuesday, Jun 27 2023'

In [27]:
current_location = lambda: "New Delhi"
current_weather = lambda: "Wonderful and raining"

In [28]:
def truncate(chatml: ChatML, max_tokens: int = 1500, keep_roles: list[str] = ["system"]) -> ChatML:
    chatml_with_idx = list(enumerate(chatml))
    chatml_to_keep = [c for c in chatml_with_idx if c[1]["role"] in keep_roles]
    budget = max_tokens - sum([
        count_tokens(to_prompt([c[1]], suffix=""))
        for c in chatml_to_keep
    ]) - count_tokens("<|section|>me (Samantha)\n")

    assert budget > 0, "keep_roles messages exhaust tokens"

    remaining = [c for c in chatml_with_idx if c[1]["role"] not in keep_roles]
    for i, c in reversed(remaining):
        c_len = count_tokens(to_prompt([c], suffix=""))
        
        if budget < c_len:
            break

        budget -= c_len
        chatml_to_keep.append((i, c))

    sorted_kept = sorted(chatml_to_keep, key=lambda x: x[0])
    return [c for _, c in sorted_kept]
    

chatml = [
    {"role": "system", "content": f"SYSTEM"},
    *[
        {"role": "user", "content": f"Hello {i}"}
        for i in range(500)
    ]
]

truncate(chatml, max_tokens=100)

[{'role': 'system', 'content': 'SYSTEM'},
 {'role': 'user', 'content': 'Hello 497'},
 {'role': 'user', 'content': 'Hello 498'},
 {'role': 'user', 'content': 'Hello 499'}]

In [41]:
# !poetry add keybert keyphrase-vectorizers 'pydantic<2'

In [45]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

vectorizer = KeyphraseCountVectorizer(pos_pattern='<N.*>')
kw_model = KeyBERT(model="all-MiniLM-L6-v2")

def extract_keywords(doc, top_n: int = 5, **kwargs) -> list[tuple[str, float]]:
    opts = {
        "top_n": top_n,
        "stop_words": "english",
        "vectorizer": vectorizer,
        "use_mmr": True,
        "diversity": 0.7,
        **kwargs,
    }
    
    keywords = kw_model.extract_keywords(doc, **opts)
    return keywords

extract_keywords("Diwank's mom's name is Shikha Singh. She is an entrepreneur and a very inspiring and brave person.")

[('diwank', 0.6332),
 ('shikha', 0.5357),
 ('singh', 0.4863),
 ('mom', 0.2978),
 ('entrepreneur', 0.1408)]

In [71]:
from pycozo.client import Client

cozo_client = Client('http', options={'host': 'http://127.0.0.1:9070'})

import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-base-v2')
model = AutoModel.from_pretrained('intfloat/e5-base-v2')

def embed(batch: list[str], instruction: str):

    input_texts = [f"{instruction}: {text}" for text in batch]
        
    # Tokenize the input texts
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    
    # (Optionally) normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings.tolist()

def get_matching_beliefs(
    chatml , #: ChatML,
    confidence: float = 0.8,
    n: int = 10,
    k: int = 2,
    window_size: int = 3,
    character_ids: list[str] = [],  # TODO
    exclude_roles: list[str] = [],
):
    # Extract text window from conversation chatml
    chatml = chatml[-window_size:]
    
    text = "\n".join([
        f"{message.get('name', message['role'])}: {message['content']}"
        for message in chatml
        if message["role"] not in exclude_roles
    ])

    # Prepare embedding for search
    radius = 1.0 - confidence
    [query_embedding] = embed([text], "passage")

    # TODO: Filter by characters
    # TODO: Apply mmr
    # Embedding search
    hnsw_query = dedent(f"""
    ?[belief, valence, dist, character_id] := ~beliefs:embedding_space{{ belief, valence, character_id |
        query: vec({query_embedding}),
        k: {n},
        ef: 20,
        radius: {radius},
        bind_distance: dist,
    }}

    :order -valence
    :order dist
    """)

    # Embedding seach results
    hnsw_results = cozo_client.run(hnsw_query)

    # Now mix up for character_ids
    groups = [
        group.tolist()
        for _, group
        in hnsw_results.groupby(["character_id"])["belief"]
    ]

    k_hnsw_results = []
    i = 0
    while len(k_hnsw_results) < k and len(groups) > 0:
        group_idx = i % len(groups)
        group = groups[group_idx]

        if not len(group):
            groups.pop(group_idx)
            continue

        item = group.pop(0)
        k_hnsw_results.append(item)
        i += 1


    # FTS query
    keywords = " ".join([
        keyword
        for keyword, _ in extract_keywords(text)
    ])
    
    fts_query = dedent(f"""
    ?[belief, valence, score, character_id] := ~beliefs:summary {{ belief, valence, character_id |
        query: "{keywords}",
        k: {n},
        score_kind: 'tf_idf',
        bind_score: score
    }}
    
    :order -valence
    :order -score
    :limit {k}
    """)

    # Embedding seach results
    fts_results = cozo_client.run(fts_query)
    fts_results = fts_results["belief"].tolist()

    # Combine results
    results = k_hnsw_results + fts_results
    
    print(f"{len(results)} matching beliefs found for ```{text}```")
    
    return results

to_belief_chatml_msg = lambda bfs: dict(role="system", name="information", content=" ".join(bfs))

chatml = [{"role": "user", "name": "Diwank", "content": "I like reading sometimes"}]
# to_belief_chatml_msg(get_matching_beliefs(chatml, 0.8))
get_matching_beliefs(chatml)

4 matching beliefs found for ```Diwank: I like reading sometimes```


['Diwank loves exploring new areas of interest.',
 "Diwank enjoys hearing other people's insights and sharing their own knowledge.",
 '- At their core, Diwank is driven by innovative ideas, products, and solutions.    - Diwank believes in being curious, open-minded, reflective, and adaptable.    - Diwank needs to visualize ambitious goals and be independent in their pursuits.',
 "Diwank doesn't care much about what others think of their shifting focus."]

In [ ]:
raise 1//0

In [74]:
diwank_pascal_beliefs = """
Pascal is both a mentor and a dear friend to Diwank.
Pascal was the first person from Silicon Valley who identified my potential and gave Diwank a chance to build things back when Diwank was only 18 years old.
Pascal invited Diwank to join a startup accelerator program called "WebFWD" that he had created when he was running Mozilla Labs.
Pascal is deeply passionate about a variety of interests and areas of expertise, such as business and physical wellness.
Pascal firmly believes in the power of technology and innovation, dedicating his career to leading and contributing to prominent tech companies.
Pascal advocates for future-oriented thinking, reflected in his co-founding of 'be radical' and authoring of 'Disrupt Disruption'.
Pascal is a strong supporter of women's empowerment, demonstrated by his co-founding of The Women's Impact Alliance.
Pascal possesses a natural instinct for mentoring, nurturing young talents and creating opportunities for them through initiatives like the WebFWD startup accelerator.
Pascal is not afraid of risk, embracing new challenges and ventures throughout his career, including running a VC fund, non-profits, and an art gallery.
Pascal values knowledge sharing and community building, contributing to platforms like TheHeretic.org and the Falling Walls Foundation.
Pascal believes strongly in the potential of young leaders, offering them guidance and support through his various advisory roles.
Pascal has a unique appreciation for physical wellness, previously working as a Pilates instructor for US Olympians.
Pascal displays humility and an eagerness to contribute, actively involving himself in various initiatives and organizations.
Pascal stands firmly for honesty and clear communication, disliking insincerity and unnecessary jargon.
Pascal finds fulfillment in educating, inspiring, and empowering others, especially leaders.
Pascal's core mission is to drive positive change and amplify the impact of other changemakers.
Pascal champions diversity and inclusion, evidenced by his establishment of The Women’s Impact Alliance.
Pascal has a penchant for disruptive thinking, always seeking to challenge existing paradigms and identify new opportunities.
Pascal places high importance on lifelong learning and continuous leadership development.
""".strip().split("\n")

# diwank_pascal_beliefs_uuids = [uuid() for _ in range(len(diwank_pascal_beliefs))]
diwank_pascal_beliefs_uuids = ['125df766-586b-4d39-9e09-01cd351a08a2',
 '8f4a0cbf-0e04-4b93-b9e8-9bcd3a3ebb22',
 'c79526fe-28c2-47a0-8baf-b01402ff7f89',
 '9517e249-5de9-4edd-b7b7-1be91dfe8b84',
 '5f2b13bd-3d72-49ee-9a57-a9a4c5c918e6',
 '4827e827-5d6a-4586-bbe7-5603eee9c39f',
 '43ce9713-8f68-4b60-990f-f11dd241fb97',
 '562a22ca-955e-4667-8f7d-876f70d52d8e',
 '3765b5ec-61f7-487e-ad28-0b29d4264f8b',
 '0293e070-9c22-45db-9120-6a5cbf2a718f',
 'd16bf2b7-2fb1-4caa-ac73-238c586a46c4',
 'd64f38cc-838c-4604-9628-e1bef7af1856',
 'aa2e30fe-b96a-4a85-83b4-ef8d861e5298',
 '5c7ce0d8-9612-4062-9394-15fcdcd279b4',
 '11f6e4b5-fd7f-4f9e-afab-0da50be8daa3',
 '638d317d-135a-42f6-b4ae-6ac536709fb5',
 '8b537ebd-ac77-4374-8e08-a077d5c31e3e',
 '7a399c3d-7640-4bf7-ac10-30f5ce1ea564',
 '6b77776c-ca66-45be-8b65-f484016b3f12']


In [76]:
for id, b in tqdm(zip(diwank_pascal_beliefs_uuids, diwank_pascal_beliefs)):
    # break

    aspects = absa(b)
    # aspects = []
    aspects_str = json.dumps(aspects)
    [e] = embed([b], "passage")
    
    query = dedent(f"""
    ?[belief_id, character_id, belief, details, parent_belief_id, valence, aspects, belief_embedding, details_embedding] <- [[
      to_uuid("{id}"),
      to_uuid("{diwank_character_id}"),
      "{rm_newline(b)}",
      "{rm_newline(b)}",
      null,
      {3.0},
      {aspects_str},
      vec({e}),
      vec({e}),
    ]]
    
    :put beliefs {{
        belief_id, character_id, belief, details, parent_belief_id, valence, belief_embedding, details_embedding
    }}
    """)

    try:
        client.run(query)
    except Exception as e:
        e = str(e)
        idx = int(re.findall(r"[\d]+", e)[0])
        print(e)
        print()
        print("="*20)
        print()
        print(query[idx-10:idx+20])

0it [00:00, ?it/s]

In [78]:
client.close()